<a href="https://colab.research.google.com/github/Areeff10/next_word_prediction/blob/main/next_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
import numpy as np
import os

In [24]:
file  = open('data.txt', 'r', encoding = "utf8")

In [25]:
lines=[]
for i in file:
    lines.append(i)

In [26]:
data=""
for i in lines:
    data = ' '.join(lines)

In [27]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [28]:
data = data.split()
data = ' '.join(data)

In [29]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [31]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [32]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [33]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [34]:
len(sequence_data)

105879

In [35]:
vocab_size = len(tokenizer.word_index)+1

In [36]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [37]:
sequence = np.array(sequence)

In [38]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [39]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [40]:
X=np.array(X)
y=np.array(y)

In [41]:
y = to_categorical(y, num_classes=vocab_size)

In [42]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000, return_sequences = True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

In [43]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [45]:
checkpoint = ModelCheckpoint("next_words.h5", monitor="loss", verbose = 1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate = 0.001))
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint])

Epoch 1/2
1653/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 5.5428
Epoch 1: loss improved from inf to 5.57745, saving model to next_words.h5


1655/1655 ━━━━━━━━━━━━━━━━━━━━ 47s 27ms/step - loss: 5.5429
Epoch 2/2
1654/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.2852
Epoch 2: loss improved from 5.57745 to 5.27201, saving model to next_words.h5


1655/1655 ━━━━━━━━━━━━━━━━━━━━ 80s 26ms/step - loss: 5.2852


In [46]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# Load tokenizer and model
tokenizer = pickle.load(open("token.pkl", "rb"))
model = load_model("next_words.h5")

def predict_next_words(model, tokenizer, text_sequence, num_words):
    for _ in range(num_words):
        # Preprocess input: tokenize and pad to length 3
        token_list = tokenizer.texts_to_sequences([text_sequence])[0]
        token_list = token_list[-3:]  # keep only last 3 tokens
        if len(token_list) < 3:
            token_list = [0] * (3 - len(token_list)) + token_list  # pad if needed
        token_array = np.array([token_list])

        # Predict the next word
        predicted_probs = model.predict(token_array, verbose=0)
        predicted_index = np.argmax(predicted_probs)

        # Convert index back to word
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                text_sequence += ' ' + word
                break
    return text_sequence


In [47]:
seed_text = "he went to"
output = predict_next_words(model, tokenizer, seed_text, num_words=5)
print(output)

he went to the door and i have


In [51]:
seed_text = "I love"
output = predict_next_words(model, tokenizer, seed_text, num_words=5)
print(output)

I love to be a little man
